## Compare binding to human versus mouse Mxra8

In [ ]:
import altair as alt

import pandas as pd

_ = alt.data_transformers.disable_max_rows()

In [ ]:
# this cell is tagged parameters for `papermill` parameterization

entry_293T_human_Mxra8 = None
binding_human_Mxra8 = None
binding_mouse_Mxra8 = None
corr_chart_html = None
min_entry_293T_human_Mxra8 = None
min_entry_293T_human_Mxra8_std = None
min_binding_std = None
min_times_seen = None

In [ ]:
# read the data

print(f"Reading cell entry from {entry_293T_human_Mxra8=}")
data_df = (
    pd.read_csv(entry_293T_human_Mxra8)
    .query("times_seen >= @min_times_seen")
    .query("effect_std <= @min_entry_293T_human_Mxra8_std")
    .assign(mutation=lambda x: x["wildtype"] + x["site"].astype(str) + x["mutant"])
    [["site", "mutation", "effect"]]
    .rename(columns={"effect": "entry_293T_human_Mxra8"})
)

for name, f in [
    ("binding_human_Mxra8", binding_human_Mxra8),
    ("binding_mouse_Mxra8", binding_mouse_Mxra8),
]:
    print(f"Reading {name=} from {f=}")
    bind_df = (
        pd.read_csv(f)
        .query("times_seen >= @min_times_seen")
        .query("frac_models == 1")
        .query("`Mxra8 binding_std` <= @min_binding_std")
        .rename(columns={"Mxra8 binding_median": name})
    )
    bind_rep_cols = bind_df.columns[11: ].tolist()
    bind_df = (
        bind_df
        .assign(
            label=lambda x: x.apply(
                lambda r: f"{r[name]:.2f} ({', '.join(str(round(r[c], 2)) for c in bind_rep_cols)})",
                axis=1,
            )
        )
        .rename(columns={"label": f"{name}_label"})
        [["mutation", name, f"{name}_label"]]
    )
    data_df = data_df.merge(
        bind_df, how="left", on=["mutation"], validate="1:1"
    )

In [ ]:
# plot the data

site_selection = alt.selection_point(on="mouseover", empty=False, fields=["site"])

mut_selection = alt.selection_point(on="mouseover", empty=False, fields=["mutation"])

min_entry_slider = alt.param(
    name="min_entry_slider",
    bind=alt.binding_range(
        min=data_df["entry_293T_human_Mxra8"].min(),
        max=0,
        name="minimum entry in 293T-human-Mxra8 cells",
    ),
    value=min_entry_293T_human_Mxra8,
)

corr_chart = (
    alt.Chart(data_df)
    .add_params(site_selection, mut_selection, min_entry_slider)
    .transform_filter(alt.datum["entry_293T_human_Mxra8"] >= min_entry_slider)
    .encode(
        alt.X("binding_human_Mxra8", title="binding human Mxra8"),
        alt.Y("binding_mouse_Mxra8", title="binding mouse Mxra8"),
        color=alt.condition(site_selection, alt.value("red"), alt.value("gray")),
        opacity=alt.condition(site_selection, alt.value(0.9), alt.value(0.15)),
        size=alt.condition(site_selection, alt.value(75), alt.value(55)),
        strokeWidth=alt.condition(mut_selection, alt.value(3), alt.value(0.7)),
        tooltip=[
            "mutation",
            alt.Tooltip("entry_293T_human_Mxra8", format=".2f", title="entry 293T-human-Mxra8"),
            alt.Tooltip("binding_human_Mxra8_label", title="binding human-Mxra8"),
            alt.Tooltip("binding_mouse_Mxra8_label", title="binding mouse-Mxra8"),
        ],
    )
    .mark_circle(stroke="black")
    .properties(width=300, height=300, title="binding to human vs mouse Mxra8")
    .configure_title(fontSize=18)
    .configure_axis(grid=False, titleFontSize=16, labelFontSize=12)
)

print(f"Saving to {corr_chart_html=}")
corr_chart.save(corr_chart_html)

corr_chart